In [1]:
#Adafruit_IO setup
from Adafruit_IO import Client, Feed, RequestError

ADAFRUIT_IO_USERNAME = "Thuviksa"
ADAFRUIT_IO_KEY = "aio_wNTx33Kk3vZccNMqPeCOSWpwIJGS"
aio = Client(ADAFRUIT_IO_USERNAME, ADAFRUIT_IO_KEY)

In [2]:
#Coordinates
metadata = {'lat': 36.017433,
           'lon': -84.239815,
           'ele': 0,
           'created_at': None}

In [3]:
#External Temperature reading
import smbus

def readTemp():
    address = 0x48
    bus = smbus.SMBus(1)
    raw = bus.read_word_data(address, 0) & 0xFFFF
    raw = ((raw << 8) & 0xFF00) + (raw >> 8)
    temperature = (raw / 32.0) / 8.0
    return temperature

In [ ]:
#Main Loop
from bme280 import BME280
import ST7735
from enviroplus.noise import Noise
from ltr559 import LTR559
import time

from IPython.display import clear_output
import busio
import board
import adafruit_ccs811

from IPython.display import clear_output
from board import *

soundFeed = aio.feeds("sound")
luxFeed = aio.feeds("light")
intTempCfeed = aio.feeds("inttempc")
intTempFfeed = aio.feeds("inttempf")
extemp2Feed = aio.feeds("extemp2")
extempFeed = aio.feeds("extemp")
humidFeed = aio.feeds("humidity")
pressFeed = aio.feeds("pressure")
tvocsFeed = aio.feeds("new-sensor.tvocs")
co2Feed = aio.feeds("new-sensor.co2")

range=[(100, 1200)]

noise = Noise()
ltr559 = LTR559()
bme280 = BME280()
i2c = board.I2C()
ccs =  adafruit_ccs811.CCS811(i2c)

Amps = noise.get_amplitudes_at_frequency_ranges(range)
Light = ltr559.get_lux()
External = readTemp()
External2 = External * 1.8 + 32
Celcius = bme280.get_temperature()
Fahrenheit = Celcius * 1.8 + 32
InternalC = Celcius - External
InternalF = Fahrenheit - External2
Humidity = bme280.get_humidity()
Pressure = bme280.get_pressure()
CO2 = ccs.eco2
TVOC = ccs.tvoc

time_calculation = time.time()
aio.send_data(co2Feed.key, CO2, metadata)
aio.send_data(tvocsFeed.key, TVOC, metadata)
aio.send_data(soundFeed.key, Amps[0], metadata)
aio.send_data(luxFeed.key, Light, metadata)
aio.send_data(intTempCfeed.key, InternalC, metadata)
aio.send_data(intTempFfeed.key, InternalF, metadata)
aio.send_data(extemp2Feed.key, External2, metadata)
aio.send_data(extempFeed.key, External, metadata)
aio.send_data(humidFeed.key, Humidity, metadata)
aio.send_data(pressFeed.key, Pressure, metadata)
time_elapsed = time.time() - time_calculation

while True:
    try:
        Amps = noise.get_amplitudes_at_frequency_ranges(range)
        Light = ltr559.get_lux()
        External = readTemp()
        External2 = External * 1.8 + 32
        InternalC = Celcius - External
        InternalF = Fahrenheit - External2
        Humidity = bme280.get_humidity()
        Pressure = bme280.get_pressure()
        CO2 = ccs.eco2
        TVOCs = ccs.tvoc
        output = "Temperature: External: {0:.2f} C, External: {1:.2f} F, Internal: {2:.2f} C, Internal: {3:.2f} F, - Humidity: {4:.2f} % - Pressure: {5:.2f} hPa, - CO2: {6:.2f} ppm, - TVOCs: {7:.2f} ppb".format(External, External2, InternalC, InternalF, Humidity, Pressure, CO2, TVOCs)
        print(output)
        time.sleep(5)
        clear_output()
        time_elapsed = time.time() - time_calculation
        if time_elapsed == 30:
            time_calculation = time.time()
            aio.send_data(co2Feed.key, CO2, metadata)
            aio.send_data(tvocsFeed.key, TVOC, metadata)
            aio.send_data(soundFeed.key, Amps[0], metadata)
            aio.send_data(luxFeed.key, Light, metadata)
            aio.send_data(intTempCfeed.key, InternalC, metadata)
            aio.send_data(intTempFfeed.key, InternalF, metadata)
            aio.send_data(extemp2Feed.key, External2, metadata)
            aio.send_data(extempFeed.key, External, metadata)
            aio.send_data(humidFeed.key, Humidity, metadata)
            aio.send_data(pressFeed.key, Pressure, metadata)
    except IOError:
        pass
    except KeyboardInterrupt:
        print("Loop terminated by user")
    except RuntimeError:
        print("Not Good")